# Ex.4 Luxembourg rGDP

Previous examples featured simulated data. Simulated data are well behaved and therefore easier to model. In this excercise we model quarterly rGDP  growth for Luxembourg (most important indicator of economic activity). This series is more difficult to model because of potential: breaks, outliers, measurement errors and is of short length. yet, modellers in (say) statistical offices, ministry of finance, etc. are required to prepare forecasts and budget projections using this series.     

In April 2022 Statec released the first estimate of rGDP for 2021Q4 (rGDP often gets revised). your task is to produce rGDP forecasts for 2022Q1 and 2022Q2.   

Tasks: 
1. Import Luxembourg rGDP series from excel file.
2. Plot the series and perform stationarity tests. 
3. Decide on the appropriate ARIMA(p,d,q) model. Inspect its properties. 
4. Forecast quarterly rGDP two periods ahead.  

In [46]:
# PART 1
# import packages
# %reset
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Import data 
#os.chdir('C:\\Users\\xct385\\Desktop\\DS ws\\MADS_win25\\Exercises\\1_Ex_UNI_TS\\Ex_models_python') # set working directory to file's location
df = pd.read_excel('rgdp_2021q4.xlsx')
df.columns = ['date', 'rgdp']


In [ ]:
# PART 2

# plots 
fig, axs = plt.subplots(1, 3,figsize=(16,8))
axs[0].plot(df.rgdp)
axs[0].set_xticks(np.arange(0,len(df.rgdp),10))                                                       # display every 10-th tick
axs[0].set_xticklabels(df.date[np.arange(0,len(df.rgdp),10)],rotation=20);                            # label ticks
sm.graphics.tsa.plot_acf(df.rgdp, lags=10, ax=axs[1]);
sm.graphics.tsa.plot_pacf(df.rgdp, lags=10, ax=axs[2]);

# test for stationarity
# ADF test
print("Results of Dickey-Fuller Test:")
res = sm.tsa.stattools.adfuller(df.rgdp,maxlag=5,regression='c',autolag='AIC')
print(f"Test Statistic:{res[0]}")
print(f"p-value:{res[1]}")
print(f"#Lags Used:{res[2]}")
print(f"#Observations Used:{res[3]}")
# KPSS test
print("Results of KPSS Test:")
res = sm.tsa.stattools.kpss(df.rgdp,regression='c',nlags='auto')
print(f"Test Statistic:{res[0]}")
print(f"p-value:{res[1]}")
print("-----------------------------------")
print("Is the series stationary?")

In [ ]:
# PART 2 - cont.
# difference the data
Y = np.asarray(df.rgdp)
y = (100*(Y[1:Y.size]/Y[0:Y.size-1]-1))   # growth rate

# plots 
fig, axs = plt.subplots(1, 3,figsize=(16,8))
axs[0].plot(y)
axs[0].set_xticks(np.arange(0,len(y),10))                                                       # display every 10-th tick
axs[0].set_xticklabels(df.date[np.arange(1,len(y),10)],rotation=20);                            # label ticks
sm.graphics.tsa.plot_acf(y, lags=10, ax=axs[1]);
sm.graphics.tsa.plot_pacf(y, lags=10, ax=axs[2]);

# test for stationarity
# ADF test
print("Results of Dickey-Fuller Test:")
res = sm.tsa.stattools.adfuller(y,maxlag=5,regression='c',autolag='AIC')
print(f"Test Statistic:{res[0]}")
print(f"p-value:{res[1]}")
print(f"#Lags Used:{res[2]}")
print(f"#Observations Used:{res[3]}")
# KPSS test
print("Results of KPSS Test:")
res = sm.tsa.stattools.kpss(y,regression='c',nlags='auto')
print(f"Test Statistic:{res[0]}")
print(f"p-value:{res[1]}")
print("-----------------------------------")
print("Is the series stationary?")
print("Can you infeer ARMA(p,q)?")


In [ ]:
# PART 3 Decide on the appropriate ARIMA(p,d,q) model
# estimate ICs
IC = sm.tsa.stattools.arma_order_select_ic(y, max_ar=4, max_ma=4, ic=['aic','bic','hqic'], trend='c')
# display optimal lags
print(f"min AIC at (p,q): {IC.aic_min_order}")
print(f"min BIC at (p,q): {IC.bic_min_order}")
print(f"min HQIC at (p,q): {IC.hqic_min_order}")
print("ICs suggest either AR(0) or AR(1).")
print("Let us estimate AR(1) and inspect its properties.")

In [ ]:
# Estimate AR(1) and comment the regression table
mod = sm.tsa.arima.ARIMA(y, order=(1, 0, 0), trend='c', validate_specification=True)
res = mod.fit()
print(res.summary())
# COMMENTS: All coeffs are statistically significant. Surprising is that the AR(1) coefficient is negative.
# Residuals seem to be free offirst order correlation (Prob(Q)=1). They do not seem to be normaly distributed (Prob(JB)=0), however, this is due to excess kurtosis.  


In [ ]:

mod = sm.tsa.arima.ARIMA(df.rgdp,np.ones((108,1)), order=(1, 1, 0))
res = mod.fit()
print(res.summary())


# Estimate AR(1) and comment the regression table
y = Y[1:Y.size]-Y[0:Y.size-1]   # growth rate

mod = sm.tsa.arima.ARIMA(y, order=(1, 0, 0))
res = mod.fit()
print(res.summary())




In [ ]:
# Plot residuals
fig, axs = plt.subplots(1, 2,figsize=(12,10))
axs[0].plot(res.resid)
axs[0].set_title("Residuals")
axs[1].hist(res.resid,bins=100)
axs[1].set_title("Histogram")

# Test for whitness of residuals 
print("Ljung-Box and Box-Pierce for residual autocorrelation")
res_lb = sm.stats.diagnostic.acorr_ljungbox(res.resid, lags=10, boxpierce=True,model_df=0+0)
print(res_lb)

# Test for heteroscedasticity
res_het = sm.tsa.stattools.breakvar_heteroskedasticity_test(res.resid, subset_length=1/3, alternative='two-sided')
print("Heteroscedasticity test results:")
print(f"P-val: {res_het[1]}")
print("---------------------------------------")
print("The model more or less satisfies the main model checking criteria.")
print("Nevertheless it pays off to estimate also neighbouring models.")

In [ ]:
# Estimate adjacent models
# Increase AR order
mod = sm.tsa.arima.ARIMA(y, order=(2, 0, 0), trend='c', validate_specification=True)
res = mod.fit()
print(res.summary())

# Add MA term
mod = sm.tsa.arima.ARIMA(y, order=(1, 0, 1), trend='c', validate_specification=True)
res = mod.fit()
print(res.summary())


In [ ]:
# PART 4 Forecast quarterly rGDP two periods ahead
# print forecasts
from lib2to3.pgen2.token import LBRACE
n_fcs = 2
n_obs = 10
fcs = res.get_forecast(steps=n_fcs)
print(fcs.summary_frame())

# plot forecasts
fig, ax = plt.subplots(figsize=(15, 5))
n = len(y)
# plot y
ax.plot(df.date.iloc[-n_obs:],y[n-n_obs:n])
# get forecast with confidence intervals
fcs = res.get_forecast(steps=n_fcs).summary_frame()
# overlay the forecast on the plot for y
fcs_mean = fcs['mean']
yT = pd.Series(y[-1:])
yTfcs = pd.concat([yT,fcs_mean], axis = 0)
ax.plot(np.arange(n_obs-1,n_obs+n_fcs),yTfcs,'k--')
# add a shaded area for confidence intervals
lb = pd.Series(fcs['mean_ci_lower'])
ub = pd.Series(fcs['mean_ci_upper'])
yTlb = pd.concat([yT,lb], axis = 0)
yTub = pd.concat([yT,ub], axis = 0)
ax.fill_between(np.arange(n_obs-1,n_obs+n_fcs), yTlb, yTub, color='k', alpha=0.1);
# update tick labels
labels = pd.concat([df.date[-n_obs:],pd.Series(['2022Q1'])],axis = 0)
labels = pd.concat([labels,pd.Series(['2022Q2'])],axis = 0)
ax.set_xticks(np.arange(0,n_obs+n_fcs))  
ax.set_xticklabels(labels,rotation=20); 

